In [ ]:
import os
import time
import azure.cosmos

db_name = "iddbtest"
container_name = "idcltest"

# Create the database if it doesn't exist
try:
    cosmos_client.create_database(db_name)
    print('Database created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/id"))
print('Container created')

In [ ]:
import urllib.request, json

# fetch and load the data
with urllib.request.urlopen("https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/indexing.json") as url:
    docs = json.loads(url.read().decode())

# insert the docs
container_client = database_client.get_container_client(container_name)
progress = 1
for doc in docs:
    if progress % 10 == 0:
        print("Inserting document: %5d / 10000\r"% (progress), end="", flush=True)
    container_client.create_item(doc)
    progress += 1

In [6]:
query = 'SELECT * FROM c WHERE c.field1 = "Mattie87"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s)
Time elapsed: 13 ms
Request charge: 2.83 RUs


In [7]:
database_client.replace_container(container_client,
    partition_key=azure.cosmos.PartitionKey(path="/id"),
    indexing_policy={
        'indexingMode': 'none'
    })
print('Indexing policy updated')

Indexing policy updated


In [9]:
query = 'SELECT * FROM c WHERE c.field1 = "Mattie87"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s)
Time elapsed: 16 ms
Request charge: 12.31 RUs


In [10]:
database_client.replace_container(container_client,
    partition_key=azure.cosmos.PartitionKey(path="/id"),
    indexing_policy={
        'indexingMode': 'consistent',
        'includedPaths': [{'path' : '/*'}],
        'excludedPaths': [{'path' : '/field2/?'}]
    })
print('Indexing policy updated')

container_client.read(populate_quota_info=True)
index_transformation_progress = container_client.client_connection.last_response_headers['x-ms-documentdb-collection-index-transformation-progress']
print('\rCurrent index transformation progress: ' + index_transformation_progress + '%', end="", flush=True)
while (index_transformation_progress != '100'):
    time.sleep(5)
    container_client.read(populate_quota_info=True)
    index_transformation_progress = container_client.client_connection.last_response_headers['x-ms-documentdb-collection-index-transformation-progress']
    print('\rCurrent index transformation progress: ' + index_transformation_progress + '%', end="", flush=True)
print('\nIndex transformation completed')

Indexing policy updated
Current index transformation progress: 100%
Index transformation completed


In [12]:
query = 'SELECT * FROM c WHERE c.field1 = "Mattie87"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s)
Time elapsed: 11 ms
Request charge: 2.83 RUs


In [13]:
query = 'SELECT * FROM c WHERE c.field2 = 3188'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 0 result(s)
Time elapsed: 12 ms
Request charge: 12.31 RUs
